In [2]:
import pandas as pd    # ура скачаем все что можно
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re
import plotly.express as px
from geopy.geocoders import Nominatim
from time import sleep
from tqdm import tqdm  # Прогресс-бар
from geopy.exc import GeocoderTimedOut

In [3]:
# Спарсим surf coffee (адреса и метро)

url = "https://www.surfcoffee.ru/surf-coffee-stores#rec76974741"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")


restaurants = soup.find_all("div", class_="t786__title t-name t-name_xl js-product-name") # тут парсим все серфы

for rest in restaurants:
    address = rest.find("div", class_="t786__descr t-descr t-descr_xxs")

    address_text = address.text.strip() if address else "Нет адреса"

restaurants_clean = []  # тут на сайте нашли, что NEO есть в названии последнего мск серфа, через поиск по слову убедились, что больше такого нет

for i in restaurants:
    restaurants_clean.append(i.text)
    if 'NEO' in i.text:
        break              # получили список мск серфов


restaurants_clean = [rest.strip().replace('®', '').replace('\xa0', ' ') for rest in restaurants_clean]


#print(len(restaurants_clean))   # 112 серфов в мск
restaurants_clean

['SOLYANKA × SURF COFFEE',
 'ARBAT × SURF COFFEE',
 'SECRET SPOT × SURF COFFEE',
 'LIPS × SURF COFFEE',
 'G-SPOT × SURF COFFEE',
 'MAYAK × SURF COFFEE',
 'SHORE × SURF COFFEE',
 'DREAM × SURF COFFEE',
 'ANTHILL X SURF COFFEE',
 'CHANGE X SURF COFFEE',
 'PKRVK X SURF COFFEE',
 'TAKEOFF X SURF COFFEE',
 'SPORT X SURF COFFEE',
 'URBAN JUNGLE X SURF COFFEE',
 'BELKA X SURF COFFEE',
 'REVOLUTION X SURF COFFEE',
 'RED VILLAGE X SURF COFFEE',
 'TRAVEL X SURF COFFEE',
 'FLAT X SURF COFFEE',
 'STUDIO X SURF COFFEE',
 'AMEN X SURF COFFEE',
 'SHINE X SURF COFFEE',
 'FLACON X SURF COFFEE',
 'EAST WEST X SURF COFFEE',
 'HIGHWAY X SURF COFFEE',
 'REPUBLIC X SURF COFFEE',
 'AVIAPARK X SURF COFFEE',
 'SABABA X SURF COFFEE',
 'AKADEMIK X SURF COFFEE',
 'OASIS X SURF COFFEE',
 'PI X SURF COFFEE',
 'LOVELY X SURF COFFEE',
 'FLOW × SURF COFFEE',
 'YOUTH × SURF COFFEE',
 'ARMA × SURF COFFEE',
 'PURE× SURF COFFEE',
 'OGNI × SURF COFFEE',
 'STARTUP × SURF COFFEE',
 'LE FORT × SURF COFFEE',
 'WINGS × SURF COF

In [4]:
# Спарсим surf coffee (адреса и метро)


address = soup.find_all("div", class_="t786__descr t-descr t-descr_xxs")

address[1:113:]
address_clean = []
for i in address[1:113:]:
    address_clean.append(i.text)
address_clean # получили адреса

# Убираем \n по краям и знак ®
address_clean = [addr.strip().replace('®', '') for addr in address_clean]


metro_list = []

# Регулярка для поиска метро (ищет "м. [название]")
metro_pattern = re.compile(r'м\.\s*([\w-]+)')

for address in address_clean:
    match = metro_pattern.search(address)
    if match:
        metro_list.append(match.group(1))  # Добавляем название метро
    else:
        metro_list.append(pd.NA)  # Если метро нет, добавляем NaN

print(metro_list) # как выяснили это все было ради 8 станций:(

print(address_clean)

['Китай-Город', 'Арбатская', 'Шаболовская', 'Академическая', 'Тургеневская', 'Маяковская', 'Мякинино', <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, 'Новокузнецкая', <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, '3', <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>]
['Москва, м. Китай-Город, ул. Солянка 1/2', 'Москва, м. Арбатская, ул. Новый Арбат 7а', 'Москва, м. Шаболовская, ул. Конный переулок д.12', 'Москва, м. Академическая, ул. Дмитрия Ульянова 4к1', 'Москва, м. Тургеневская, ул. Мясницкая 16', 'Москва, м. Маяков

In [5]:
# Сделаем датафрейм c surf coffee
address_clean = [re.sub(r"Москва(?=[^ ,])", "Москва ", address) for address in address_clean]
df = pd.DataFrame({'Названия': restaurants_clean, 'Адрес': address_clean, 'Метро': metro_list})
display(df)

,Названия,Адрес,Метро
0,SOLYANKA × SURF COFFEE,"Москва, м. Китай-Город, ул. Солянка 1/2",Китай-Город
1,ARBAT × SURF COFFEE,"Москва, м. Арбатская, ул. Новый Арбат 7а",Арбатская
2,SECRET SPOT × SURF COFFEE,"Москва, м. Шаболовская, ул. Конный переулок д.12",Шаболовская
3,LIPS × SURF COFFEE,"Москва, м. Академическая, ул. Дмитрия Ульянова...",Академическая
4,G-SPOT × SURF COFFEE,"Москва, м. Тургеневская, ул. Мясницкая 16",Тургеневская
...,...,...,...
107,GRAMMY X SURF COFFEE,Москва Варшавское ш. вл. 150б/1,<NA>
108,SUNDAY X SURF COFFEE,Москва Авиаторов 1,<NA>
109,BOVE X SURF COFFEE,Москва Академика Сахарова 7,<NA>
110,SRETENKA X SURF COFFEE,Москва ул. Сретенка 27с1,<NA>


In [6]:
# ГПТ сказал, что можно не парсить координаты, а с помощью геокодера сразу их узнать) ура-ура


geolocator = Nominatim(user_agent="coffee_map")

# Создаём новые столбцы для координат
df["Latitude"] = None
df["Longitude"] = None

# Проходим по всем адресам и получаем координаты
for i, row in df.iterrows():
    try:
        location = geolocator.geocode(row["Адрес"])
        if location:
            df.at[i, "Latitude"] = location.latitude
            df.at[i, "Longitude"] = location.longitude
    except Exception as e:
        print(f"Ошибка с адресом {row['Адрес']}: {e}")
    sleep(1)


# Строим карту
fig = px.scatter_mapbox(
    df,
    lat="Latitude",
    lon="Longitude",
    text="Названия",
    zoom=10,
    center={"lat": 55.7558, "lon": 37.6173},  # Центр Москвы
    mapbox_style="open-street-map"
)

fig.show() #  чет мало что отобразилось:(
           # UPD: теперь нормально:)


Ошибка с адресом ул. 1-ая Тверская-Ямская, 10: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D1%83%D0%BB.+1-%D0%B0%D1%8F+%D0%A2%D0%B2%D0%B5%D1%80%D1%81%D0%BA%D0%B0%D1%8F-%D0%AF%D0%BC%D1%81%D0%BA%D0%B0%D1%8F%2C+10&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Ошибка с адресом Москва Ленинградский пр-т 36 стр. 38: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0+%D0%9B%D0%B5%D0%BD%D0%B8%D0%BD%D0%B3%D1%80%D0%B0%D0%B4%D1%81%D0%BA%D0%B8%D0%B9+%D0%BF%D1%80-%D1%82+36+%D1%81%D1%82%D1%80.+38&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Ошибка с адресом Москва Дмитровское ш. 107А к1: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0+%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%BE%D0%B2%D1%81%D0%BA%D0%BE%D0%B5+%D1%88.+107%D0%90+%D0%BA1&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Ошибка с адресом Москва Тверская 4: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0+%D0%A2%D0%B2%D0%B5%D1%80%D1%81%D0%BA%D0%B0%D1%8F+4&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [17]:
url2 = "https://luchshie-dostavki.ru/restaurant/surf-coffee-x-element_73343/menu" # нашли какой-то сайт с доставкой из серфа, потому что у них а сайте цен нет
headers = {"User-Agent": "Mozilla/5.0"}

response2 = requests.get(url2, headers=headers)
soup2 = BeautifulSoup(response2.text, "html.parser")


dishes = soup2.find_all("div", class_="h6 mt-1") # тут парсим все серфы
prices = soup2.find_all("div", class_="h6 text-dark")

dishes_clean = []
prices_clean = []

for di in dishes:
    dishes_clean.append(di.text)

for pr in prices:
    prices_clean.append(pr.text)

dishes_clean = [item.strip() for item in dishes_clean]
prices_clean = [item.strip() for item in prices_clean]
prices_clean = [int(re.sub(r"\D+", "", price)) for price in prices_clean] # убрали "руб." и сделали числами


df_surf_menu = pd.DataFrame({'Блюдо': dishes_clean, 'Цена (руб.)': prices_clean})
display(df_surf_menu)

from google.colab.data_table import DataTable # ГПТ подсказал как полностью посмотреть на датафрейм

DataTable(df_surf_menu)

,Блюдо,Цена (руб.)
0,Капучино Grand,380
1,Капучино Medium,345
2,Пончик лимонно-маковый,250
3,Латте Medium,345
4,Сэндвич 4 сыра,440
...,...,...
120,Батончик ореховый с маслом CBD,260
121,Дрип Кофе Kenya Tatu,450
122,Кофе зерновой Kenya Okland,900
123,Кофе зерновой Burundi Nabassa,900


,Блюдо,Цена (руб.)
0,Капучино Grand,380
1,Капучино Medium,345
2,Пончик лимонно-маковый,250
3,Латте Medium,345
4,Сэндвич 4 сыра,440
...,...,...
120,Батончик ореховый с маслом CBD,260
121,Дрип Кофе Kenya Tatu,450
122,Кофе зерновой Kenya Okland,900
123,Кофе зерновой Burundi Nabassa,900


In [20]:
# Найдем среднюю цену напитков 0 1 3 4 52-62 77-116 - индексы напитков
indexes = [0, 1, 3, 4] + list(range(52, 63)) + list(range(77, 117))
average_surf_price_beverage = float(np.mean([prices_clean[i] for i in indexes]))
average_surf_price_beverage  # средняя цена напитка в surf_cofee - 391 рубль

391.0